In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Mayank\\Desktop\\extras\\Advanced ML Projects\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Mayank\\Desktop\\extras\\Advanced ML Projects\\Kidney-Disease-Classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mayank2393/Kidney-Disease-Classification-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mayank2393"
os.environ["MLFLOW_TRACKING_PASSWORD"]="decd1bff44b774a7f18d6d0058cf555d97ac87f9"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from Kidney_Disease_Classification.constants import *
from Kidney_Disease_Classification.utils.common import read_yaml,create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/mayank2393/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
# import dagshub

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    # def log_into_mlflow(self):
    #     # Initialize DagsHub with your repo owner and repo name
    #     dagshub.init(repo_owner='mayank2393', repo_name='Kidney-Disease-Classification')
    #     mlflow.set_tracking_uri("https://dagshub.com/mayank2393/Kidney-Disease-Classification-MLflow-DVC.mlflow")
    #     # Start an MLflow run
    #     with mlflow.start_run():
    #         # Log parameters
    #         mlflow.log_params(self.config.all_params)

    #         # Log metrics
    #         mlflow.log_metrics(
    #             {"loss": self.score[0], "accuracy": self.score[1]}
    #         )

    #         # Register the model
    #         mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")

# Example configuration structure

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    # evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-11-07 00:19:24,326: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-07 00:19:24,340: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-07 00:19:24,343: INFO: common: created directory at: artifacts]
Found 1936 images belonging to 2 classes.
121/121 [==============================] - 120s 990ms/step - loss: 1.3335 - accuracy: 0.8564
[2024-11-07 00:21:25,482: INFO: common: json file saved at: scores.json]
